<a href="https://colab.research.google.com/github/ambulesubhada07/Speech_learning/blob/main/Speech_Audio_Based_Question_Answering_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 15.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import pandas as pd
import torch
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer, WhisperProcessor, WhisperForConditionalGeneration
import speech_recognition as sr


In [ ]:
data_path = "/content/data (1).csv"
data = pd.read_csv(data_path)

In [ ]:
data.head(10)

,Pos,Player,Team,Span,Innings,Runs,Highest Score,Average,Strike Rate
0,1,Sachin Tendulkar,India,1989-2012,452,18426,200,44.83,86.23
1,2,Kumar Sangakkara,Sri Lanka,2000-2015,380,14234,169,41.98,78.86
2,3,Ricky Ponting,Australia,1995-2012,365,13704,164,42.03,80.39
3,4,Sanath Jayasuriya,Sri Lanka,1989-2011,433,13430,189,32.36,91.20
4,5,Mahela Jayawardene,Sri Lanka,1998-2015,418,12650,144,33.37,78.96
5,6,Virat Kohli,India,2008-2020,236,11867,183,59.85,93.39
6,7,Inzamam-ul-Haq,Pakistan,1991-2007,350,11739,137,39.52,74.24
7,8,Jacques Kallis,South Africa,1996-2014,314,11579,139,44.36,72.89
8,9,Saurav Ganguly,India,1992-2007,300,11363,183,41.02,73.70
9,10,Rahul Dravid,India,1996-2011,318,10889,153,39.16,71.24


In [ ]:
# Load the QA model
qa_model_name = "deepset/roberta-base-squad2"    #Uses a pre-trained RoBERTa model.
tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)  #Loads the actual question-answering model.
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Load Whisper model for speech-to-text
whisper_model = "openai/whisper-small"    # Uses OpenAI's Whisper model for speech-to-text conversion.
whisper_processor = WhisperProcessor.from_pretrained(whisper_model)    #Prepares the audio for model processing.
whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model)  #Loads the Whisper model for speech recognition.

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()             #process audio input.
    with sr.AudioFile(audio_file) as source:  #Load the audio file
        audio = recognizer.record(source)     # capture the entire audio for processing.
    try:                                           #Convert Audio to Text
        return recognizer.recognize_google(audio)    #Returns the transcribed text if successful.
    except sr.UnknownValueError:                     #Handle Errors
        return "Sorry, could not understand the audio."
    except sr.RequestError:                           #RequestError: If there's a connection issue, it informs the user.
        return "Could not request results, check your internet connection."

In [ ]:
def answer_question(question):   #Convert the DataFrame into a Single Text Context
    """Answer questions based on the tabular data."""
    context = " ".join(data.astype(str).apply(lambda x: ' '.join(x), axis=1))
    result = qa_pipeline(question=question, context=context)   #Use Hugging Face's QA Model to Find an Answer
    return result['answer']   #Return the Answer

In [ ]:
def process_audio(audio_file):
    text = transcribe_audio(audio_file)   # convert spoken words into text.
    if "Sorry" in text:                    # Handle Errors in Speech Recognition
        return text
    answer = answer_question(text)          # Answer the Question
    return f"Question: {text}\nAnswer: {answer}"   #Return the Final Answer

In [ ]:
gui = gr.Interface(     # Breakdown of Parameters
    fn=process_audio,
    inputs=gr.Audio("microphone", type="filepath"),
    outputs="text",
    title="Speech-to-Text Question Answering on Tabular Data",
    description="Ask a question by speaking, and get an answer from the CSV data."
)

gui.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f30f2ddc9b3dce917.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
